In [22]:
import sys
import os


project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [23]:
from src.prod_plan import build_smarter_production_plan
from src.transport_truck import simulate_truck_allocation_pandas
from src.deplyment import accurate_deployment

In [24]:
import polars as pl 
import numpy as np
from datetime import timedelta, datetime
import pandas as pd

rng = np.random.default_rng(seed=42)
horizon = 45

demand_outlook = pl.read_csv("Symulation data/demand_outlook.csv")
prod_master = pl.read_csv("Symulation data/prod_master.csv")
Prod_req = pl.read_csv("Symulation data/Prod_req_df.csv")
df_tt = pl.read_csv("Symulation data/lane_rules.csv")
pal_size_df = pl.read_csv("Symulation data/pal_size_df.csv") 

In [25]:
# Aggregate all store demands to plant level

plant_demand = (Prod_req
    .group_by(["plant_id", "item_id", "day", "date"])
    .agg([
        pl.sum("prod_requirements").alias("total_daily_requirement"),
        pl.max("current_stockout_risk").alias("has_stockout_risk"),
        pl.max("current_ss_risk").alias("has_ss_risk"),
        pl.first('starting_inventory_plant').alias('starting_inventory_plant')
    ])
    .sort(["plant_id", "item_id", "day", "date"])
)

# Join with production constraints
plant_plan = (plant_demand
    .join(prod_master, on=["plant_id", "item_id"])
    .with_columns([
        # Priority logic
        pl.when(pl.col("has_stockout_risk") > 0)
        .then(1)  # Prio 1: Stockout
        .when(pl.col("has_ss_risk") > 0)  
        .then(2)  # Prio 2: Safety stock risk
        .otherwise(3)  # Prio 3: Normal
        .alias("priority")
    ])
)

In [26]:
# Convert to pandas for processing
plant_plan_pd = plant_plan.to_pandas()

results = []
for (plant_id, item_id), group in plant_plan_pd.groupby(['plant_id', 'item_id']):
    processed_group = build_smarter_production_plan(group)
    results.append(processed_group)

# Combine results and convert back to Polars
production_plan_pd = pd.concat(results, ignore_index=True)
production_plan = pl.from_pandas(production_plan_pd).sort(["plant_id", "item_id", "day"])

production_plan.write_csv("Data/Prod_Plan_Before.csv")

In [27]:
deployments = accurate_deployment(production_plan, demand_outlook)

In [28]:
# --- Step 1: build pallets summary ---
deployment_with_pallets = deployments.join(
    pal_size_df, on="plant_id", how="left"
)

deployment_with_pallets = deployment_with_pallets.with_columns([
    (pl.col("quantity") / pl.col("pallet_size")).alias("pallet_exact")
])

daily_pallet_summary = (
    deployment_with_pallets
    .group_by(["day", "plant_id", "item_id", "store_id", "priority", "pallet_size"])
    .agg([
        pl.col("pallet_exact").sum().alias("store_pallet_total"),
        pl.col("quantity").sum().alias("deployment_qty")
    ])
    .sort(["day", "plant_id", "store_id", "item_id", "priority"])
)
# --- move from Polars → Pandas ---
daily_pallet_summary_pd = daily_pallet_summary.to_pandas()


In [29]:
out_pd = simulate_truck_allocation_pandas(daily_pallet_summary_pd, truck_capacity=34.0)

out_pl = pl.from_pandas(out_pd)

In [30]:
out_pl = (
    out_pl.group_by(["day", "plant_id", "store_id", "item_id", "pallet_size"])
    .agg([pl.col("qty_sent").sum().alias("qty_sent"),
          pl.col("pallets_sent").sum().alias("pallets_sent")])
)

In [31]:
out_pl.write_csv("Data/Deployment_Before.csv")